# PostgreSQL Data Governance Demonstration
This notebook demonstrates how to implement **data governance** principles using PostgreSQL. It covers user management, access control, and best practices for securing sensitive data.

We will work with a sample **patients database**, ensuring compliance with privacy and security regulations.


## Database Connection Setup
First, let's set up the connection to our PostgreSQL database.

In [ ]:
from sqlalchemy import create_engine
import psycopg2

# PostgreSQL connection settings
PG_ADDR = 'localhost'  # Server address
PG_PORT = '5432'       # PostgreSQL port
PG_USER = 'postgres'   # Admin user
PG_PASW = 'secure_password'  # Secure password
PG_DBNA = 'healthcare_db'  # Database name

# Connect to the default PostgreSQL database to check and create if needed
default_engine = create_engine(f'postgresql://{PG_USER}:{PG_PASW}@{PG_ADDR}:{PG_PORT}/postgres')

with default_engine.connect().execution_options(autocommit=True) as conn:
    result = conn.execute("SELECT 1 FROM pg_database WHERE datname = %s;", (PG_DBNA,))
    exists = result.scalar()
    
    if not exists:
        conn.execute(f"CREATE DATABASE {PG_DBNA};")
        print(f"Database '{PG_DBNA}' created successfully!")

# Now connect to the newly created database
engine = create_engine(f'postgresql://{PG_USER}:{PG_PASW}@{PG_ADDR}:{PG_PORT}/{PG_DBNA}')
print(f"Connected to database '{PG_DBNA}'.")

## Creating the Patients Table
Let's create a new table to store patient information. This table includes sensitive data, so we will ensure proper access control.

In [ ]:
with engine.connect().execution_options(autocommit=True) as conn:
    conn.execute('''DROP TABLE IF EXISTS patients;''')
    conn.execute('''
        CREATE TABLE patients (
            id SERIAL PRIMARY KEY,
            full_name VARCHAR(100),
            date_of_birth DATE,
            country VARCHAR(50),
            diagnosis VARCHAR(100),
            insurance_amount DECIMAL(10,2)
        );
    ''')
print("Table 'patients' created successfully!")

## Inserting Sample Patient Data
Now, we will populate the **patients** table with fictitious data.

In [ ]:
with engine.connect().execution_options(autocommit=True) as conn:
    conn.execute("INSERT INTO patients (full_name, date_of_birth, country, diagnosis, insurance_amount) VALUES ('Ashley Bass', '1943-08-20', 'Uruguay', 'Hypertension', 2067.78);")
    conn.execute("INSERT INTO patients (full_name, date_of_birth, country, diagnosis, insurance_amount) VALUES ('Robert Fox', '1996-06-12', 'Iraq', 'Cancer', 2531.34);")
    conn.execute("INSERT INTO patients (full_name, date_of_birth, country, diagnosis, insurance_amount) VALUES ('Monica Douglas', '1931-03-21', 'Gibraltar', 'Hypertension', 3910.82);")
    conn.execute("INSERT INTO patients (full_name, date_of_birth, country, diagnosis, insurance_amount) VALUES ('Vanessa Monroe', '1992-07-27', 'Sierra Leone', 'Diabetes', 4399.5);")
    conn.execute("INSERT INTO patients (full_name, date_of_birth, country, diagnosis, insurance_amount) VALUES ('Joel Parker', '1977-01-08', 'American Samoa', 'Diabetes', 2419.18);")
    conn.execute("INSERT INTO patients (full_name, date_of_birth, country, diagnosis, insurance_amount) VALUES ('Gabriel Jones', '1973-03-24', 'Lithuania', 'Diabetes', 2228.6);")
    conn.execute("INSERT INTO patients (full_name, date_of_birth, country, diagnosis, insurance_amount) VALUES ('David Barnes', '1977-01-09', 'Mali', 'Cancer', 3369.18);")
    conn.execute("INSERT INTO patients (full_name, date_of_birth, country, diagnosis, insurance_amount) VALUES ('Amanda Brown', '2017-05-23', 'Russian Federation', 'Diabetes', 4962.82);")
    conn.execute("INSERT INTO patients (full_name, date_of_birth, country, diagnosis, insurance_amount) VALUES ('Monica Rodriguez', '1932-10-12', 'Marshall Islands', 'Diabetes', 1331.74);")
    conn.execute("INSERT INTO patients (full_name, date_of_birth, country, diagnosis, insurance_amount) VALUES ('Hayley Zamora', '1967-08-08', 'Guyana', 'Hypertension', 939.9);")
print("Sample patient data inserted successfully!")

## Managing User Access
We will now create three new users with different roles and apply access restrictions.

In [ ]:
users = ['nurse', 'consultant', 'doctor']

with engine.connect().execution_options(autocommit=True) as conn:
    for user in users:
        conn.execute(f"DROP USER IF EXISTS {user};")
        conn.execute(f"CREATE USER {user} WITH PASSWORD 'password123';")
print("Users created successfully!")

## Revoking Privileges from Admin User
To enforce stricter data governance, we will revoke all privileges on the `patients` table from the **postgres** admin user.

In [ ]:
with engine.connect().execution_options(autocommit=True) as conn:
    conn.execute("REVOKE ALL PRIVILEGES ON TABLE patients FROM postgres;")
print("Privileges revoked from 'postgres' user!")

## Verifying Permissions
Let's check which users currently have access to the `patients` table.

In [ ]:
with engine.connect() as conn:
    result = conn.execute(
        "SELECT grantee, privilege_type FROM information_schema.role_table_grants "
        "WHERE table_name = 'patients';"
    )
    for row in result:
        print(row)
print("Current access privileges displayed!")